# callPycheronMetric Tutorial 

The goal of this tutorial is to show users how to run a dataset through the callPycheronMetric script. We will walk through:

1. [Installing Pycheron](#pycheron-install)
2. [Loading Datasets](#dataset)
3. [Running Pycheron](#running-callpycheronmetric)
4. [Outputs](#outputs)

## 1. Pycheron Install

Pycheron requires several python libraries to run as well as a `gfortran` library, if the user plans on using the fortran feature. 


### Download
Download Pycheron from the [gitlab repository](https://gitlab.sandia.gov/lynm/pycheron) and change into the top-level `pycheron` directory. You will see a `build.sh` file.

### Easiest

The easiest way to install pycheron is to run:

`sh build.sh <pycheron-version>`

This will uninstall any old versions of pycheron and install all the python dependencies.

### Build from scratch

To build the version from scratch run from the top-level directory: 

`python setup.py sdist bdist_wheel`

and then

`cd dist`

from there you can use `pip` to install the `.whl` file:

`pip install pycheron-$version-py2-none-any.whl`

pisces will need to be installed separately

`pip install pisces==0.2.4`

### gfortran

For Linux and OSX users, the option to use fortran-based scripts is available and recommended for faster runtimes. In order to use fortran a compiler needs to be installed. It is recommended that `gfortran` be used. For MacOS `gfortran 6.3` is specifically recommended and can be found [here](https://github.com/fxcoudert/gfortran-for-macOS/releases). For Unix systems gfortran binaries can be found [here](https://gcc.gnu.org/wiki/GFortranBinaries)

### Know install issues

- `obspy 1.1.0` has been release and updated their module taup, which is used by `pisces`. An `ImportError` my occur. The fix is on line 24 of `picses util.py`, replace:

  `from obspy.taup import taup`

   to

  `from obspy import taup`

     A issue has been submited on github to have this bug fixed in `pisces 0.2.4`. (https://github.com/jkmacc-LANL/pisces/issues/28)


- `Numpy 1.14.5` is recommended to use if using Fortran. Higher versions have a bug in   `f2py`

## 2. Datasets

`callPycheronMetric` accepts four data types:

- Directory (of MSEED files)
- Wfdisc (CSS)
- Obspy Stream
- Singular MSEED file

Depending on the data ingested there are different parameters that can be set. These will be gone over in Section 3.

### Directory

If a directory is being used, pycheron assumes that all files are `.mseed` files. The folder structure should be:

```
My Folder
    BGU_EHE_001.mseed
    BGU_EHN_001.mseed
    BGU_EHE_002.mseed
    BGU_EHN_002.mseed
    JPU_EHE_001.mseed
    JPU_EHN_001.mseed
    JPU_EHE_002.mseed
    JPU_EHN_002.mseed
    TCRU_EHE_001.mseed
    TCRU_EHN_001.mseed
    TCRU_EHE_002.mseed
    TCRU_EHN_002.mseed
```

With each stream name in the format `<station>_<channel>_<julian day>.mseed`. The data used in this tutorial is already formatted correctly, however if you need to format your own data you can use the pycheron function `format_mseed_filenames` in the `pycheron.util.format` package.

### Wfdisc (CSS)

If a Wfidisc (CSS) file is being used the only file that is input into the function is the actually `.wfdisc` file. However, you do need to make sure the `.w` paths listed in the `.wfdisc` file are all relative to the file. For example:

```
My Folder
    data.wfdisc
    data_directory
        OWUT_EHZ_001.w 
        OWUT_EHZ_002.w 
        OWUT_EHZ_003.w 
        .
        .
        .
        waveformN.w
```

and inside the `data.wfdisc`:

```
OWUT   EHZ       1293840000.00000   1927888      774  2011001  1293860963.29000  2096330  100.000000          1.00000         -1.00000 -      o i4 - data_directory               OWUT_EHZ_001.w                            0        -1 10-APR-18        
OWUT   EHZ       1293861020.31000   1927890      774  2011001  1293861041.29000     2099  100.000000          1.00000         -1.00000 -      o i4 - data_directory               OWUT_EHZ_002.w                      8385320        -1 10-APR-18        
OWUT   EHZ       1293861041.41000   1927892      774  2011001  1293865831.39000   478999  100.000000          1.00000         -1.00000 -      o i4 - data_directory               OWUT_EHZ_003.w                      8393716        -1 10-APR-18        
```

### mseed

This is a single `mseed` file which is read in. It is suggested you use the directory and split the data up if the single stream contains multiple days.

### stream

This is to be used only if inside a python console and not running via terminal. It is assumed that the stream has been either manually loaded using `obspy.read()` or downloaded using the obspy client and then calling the `callPycheronMetric()` as a function

### Connecting to a database with Wfdisc tables

Pycheron currently supports connection to wfdisc databases via SQLAlchemy.

## 3. Running callPycheronMetric

### Parameters

The `callPycheronMetric` has over 50 parameters, however the majority of them are all the thresholds for each of the corresponding metrics. What `callPycheronMetric` actually does is combine all of the metrics in the `pycheron.metrics` package and runs them, puts them into a database or csv, and plots figures. It is recommended you throughly read the documentation page on `callPycheronMetric` as it explains every single parameter. 

### Running via Command Line

The easiest way to run `callPycheronMetric` is via the command line. Is by using a `config.yaml` file. There is a sample template in the `pycheron` folder that defines all of the default vaules. For this tutorial you can find it in the `data` folder.

### Running via Python

## Outputs

### CSV

Output to a CSV file is possible. The output is saved into the following folder structure: 

```
<output_directory>
    <network_dir>
        <station_dir>
            <channel_1_dir>
                <metric>.csv
                <metric_plot>.png
            <channel_2_dir>
                <metric>.csv
                <metric_plot>.png
            <station_level_metrics>.csv
            <station_level_plot>.png
         <network_level_metric>.csv
         <network_level_plot>.png
```

### Database

If desired pycheron can output results into a sqllite database locally on you system. The default name for the database is `pycheron.db` but it can chnaged in the `config.yaml`

### Pycheron UI

To run the UI, from the pycheron folder call:

`python ../pycheron/UI/createDashUI.py`

This should bring up another window with the UI.

## 4. callPycheronMetric Example

The `callPycheronMetric.py` script is installed as a script so you are able to run it directly from the command line

In [1]:
%%bash

python ../pycheron/callPycheronMetric.py data/callPycheronMetric_tutorial_config.yaml

-----------------------------------------------
Beginning Metric Calculations for UU.BGU Jul Date: 001
-----------------------------------------------
Calculating PSDS. This could take a while...
No results found
No results found
No results found
callPycheron(): Finished psdMetric for: UU.BGU
callPycheron(): correlationMetric - No matching Traces... Skipping
No results found
callPycheron(): Finished gapMetric for: UU.BGU
callPycheron(): transferFunctionMetric: No Matching Traces... Skipping Metric
No results found
No results found
No results found
callPycheron(): Finished basicStatsMetric for: UU.BGU
No results found
No results found
No results found
callPycheron(): Finished DCOffset for:UU.BGU
No results found
No results found
No results found
callPycheron(): Finished staltaMetric for: UU.BGU
No results found
No results found
No results found
callPycheron(): Finished snrMetric for: UU.BGU
No results found
No results found
No results found
callPycheron(): Finished deadChannelMetric for

Traceback (most recent call last):
  File "../pycheron/callPycheronMetric.py", line 1703, in <module>
    main()
  File "../pycheron/callPycheronMetric.py", line 1696, in main
    to_csv=cfg["to_csv"], stationStartAt=cfg["stationStartAt"])
  File "../pycheron/callPycheronMetric.py", line 811, in callPycheron
    dcLinThresholdHour, byHourOn, db, session)
  File "../pycheron/callPycheronMetric.py", line 1094, in _call_pycheron_wrapper
    network=network, station=station, channel=None, session=session)
  File "/Users/jbobeck/anaconda2/lib/python2.7/site-packages/pycheron/plotting/dailyPdfPlot.py", line 130, in dailyPdfplots
    location=location, logger=logger)
  File "/Users/jbobeck/anaconda2/lib/python2.7/site-packages/pycheron/plotting/dailyPdfPlot.py", line 673, in _pdf_plots_from_database
    stats = psdStatistics(chan_psds, logger=logger, database=database)
  File "/Users/jbobeck/anaconda2/lib/python2.7/site-packages/pycheron/psd/psdStatistics.py", line 267, in psdStatistics
    d